In [1]:
"""
log(SalePrice)



Feature Engineering Summary:

1. YrSold, MoSold
TimeSold = YrSold + (MoSold - 1)/12
or
TimeSoldSin = sin(2π * MoSold/12)
TimeSoldCos = cos(2π * MoSold/12)

2. YearRemodAdd, YearBuilt, GarageYrBlt -> Age, RemodAge, GarageAge
Age = YrSold - YearBuilt
RemodAge = YrSold - YearRemodAdd
GarageAge = YrSold - GarageYrBlt

Filling NAs:
Numerical columns: filled with median values
Categorical columns: filled with mode values
Categorical Encoding: One-Hot Encoding

Feature Engineering:
GarageCars, GarageArea: Both features indicate the garage size
HouseSize: TotalBsmtSF, 1stFlrSF, 2ndFlrSF combined
Bsmt1stSF: TotalBsmtSFm, 1stFlrSF combined

Model: 
Random Forest
Gradient Boosting
"""

'\nlog(SalePrice)\n\n\n\nFeature Engineering Summary:\n\n1. YrSold, MoSold\nTimeSold = YrSold + (MoSold - 1)/12\nor\nTimeSoldSin = sin(2π * MoSold/12)\nTimeSoldCos = cos(2π * MoSold/12)\n\n2. YearRemodAdd, YearBuilt, GarageYrBlt -> Age, RemodAge, GarageAge\nAge = YrSold - YearBuilt\nRemodAge = YrSold - YearRemodAdd\nGarageAge = YrSold - GarageYrBlt\n\nFilling NAs:\nNumerical columns: filled with median values\nCategorical columns: filled with mode values\nCategorical Encoding: One-Hot Encoding\n\nFeature Engineering:\nGarageCars, GarageArea: Both features indicate the garage size\nHouseSize: TotalBsmtSF, 1stFlrSF, 2ndFlrSF combined\nBsmt1stSF: TotalBsmtSFm, 1stFlrSF combined\n\nModel: \nRandom Forest\nGradient Boosting\n'

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [3]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [4]:
X = train.drop(columns=['Id', 'SalePrice'])
y = np.log1p(train['SalePrice'])
X_test = test.drop(columns=['Id'])

In [5]:
# Handle missing values
num_cols = X.select_dtypes(include=['float64', 'int64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

def fill_missing(train_df, val_df, test_df=None):
    train_df = train_df.copy()
    val_df = val_df.copy()
    test_df = test_df.copy() if test_df is not None else None

    # numeric: median
    med = train_df[num_cols].median()
    train_df[num_cols] = train_df[num_cols].fillna(med)
    val_df[num_cols] = val_df[num_cols].fillna(med)
    if test_df is not None:
        test_df[num_cols] = test_df[num_cols].fillna(med)

    # categorical: mode
    mode = train_df[cat_cols].mode().iloc[0]
    train_df[cat_cols] = train_df[cat_cols].fillna(mode)
    val_df[cat_cols] = val_df[cat_cols].fillna(mode)
    if test_df is not None:
        test_df[cat_cols] = test_df[cat_cols].fillna(mode)

    return (train_df, val_df, test_df) if test_df is not None else (train_df, val_df)

In [6]:
def add_features(df):
    df = df.copy()

    # Feature experiment: AreaPerCar
    df["AreaPerCar"] = df["GarageArea"] / (df["GarageCars"] + 1)

    # Feature experiment: HasGarage
    df["HasGarage"] = (df["GarageCars"] > 0).astype(int)

    # Feature experiment: Bsmt1stSF
    # df["Bsmt12stSF"] = df["TotalBsmtSF"] + df["1stFlrSF"] + df["2ndFlrSF"]
    df["Bsmt1stSF"] = df["TotalBsmtSF"] + df["1stFlrSF"]

    # Feature experiment: TimeSold
    df["TimeSold"] = df["YrSold"] + (df["MoSold"] - 1) / 12

    # Year diffs (Age features)
    # df["Age"] = df["YrSold"] - df["YearBuilt"]
    # df["RemodAge"] = df["YrSold"] - df["YearRemodAdd"]
    # df["GarageAge"] = df["YrSold"] - df["GarageYrBlt"]

    df["Age"] = df["TimeSold"] - df["YearBuilt"]
    df["RemodAge"] = df["TimeSold"] - df["YearRemodAdd"]
    df["GarageAge"] = df["TimeSold"] - df["GarageYrBlt"]
    return df

In [7]:
def onehot_align(train_df, val_df, test_df=None):
    train_oh = pd.get_dummies(train_df)
    val_oh = pd.get_dummies(val_df)

    val_oh = val_oh.reindex(columns=train_oh.columns, fill_value=0)

    if test_df is not None:
        test_oh = pd.get_dummies(test_df)
        test_oh = test_oh.reindex(columns=train_oh.columns, fill_value=0)
        return train_oh, val_oh, test_oh

    return train_oh, val_oh

In [8]:
# Optionally drop used cols
def drop_used_cols(df):
    # df = df.copy()
    # df = df.drop(columns=['TimeSold'])
    return df

In [9]:
model_configs = {
    "rf": [
        {
            "n_estimators": 300,
            "max_features": None
        },
        {
            "n_estimators": 300,
            "max_features": "sqrt"
        }
    ],
    "gbr": [
        {
            "n_estimators": 400,
            "learning_rate": 0.05
        },
        {
            "n_estimators": 700,
            "learning_rate": 0.05
        },
        {
            "n_estimators": 400,
            "learning_rate": 0.03
        }
    ]
}

In [10]:
kf = KFold(n_splits=5, shuffle=True, random_state=30)

cv_results = [] 


for model_name, param_list in model_configs.items(): 
    for params in param_list: 
        if model_name == "rf":
            model = RandomForestRegressor(
                random_state=30,
                n_jobs=-1,
                **params
            )
        elif model_name == "gbr":
            model = GradientBoostingRegressor(
                random_state=30,
                **params
            )

        rmses = []
        oof_pred = np.full(len(X), np.nan) 

        for train_idx, val_idx in kf.split(X):
            X_train = X.iloc[train_idx]
            X_val   = X.iloc[val_idx]
            y_train = y.iloc[train_idx]
            y_val   = y.iloc[val_idx]

            # Missing values
            X_train, X_val = fill_missing(X_train, X_val)

            # Feature engineering
            X_train = add_features(X_train)
            X_val   = add_features(X_val)

            # Drop used columns
            X_train = drop_used_cols(X_train)
            X_val   = drop_used_cols(X_val)

            # One-hot + align
            X_train, X_val = onehot_align(X_train, X_val)

            # Fit / predict
            model.fit(X_train, y_train)
            pred = model.predict(X_val)
            oof_pred[val_idx] = pred
            rmses.append(root_mean_squared_error(y_val, pred))

        mean_rmse = float(np.mean(rmses))
        std_rmse  = float(np.std(rmses))

        cv_results.append({ 
            "model": model_name,
            "params": params,
            "mean_rmse": mean_rmse,
            "std_rmse": std_rmse,
            "oof_pred": oof_pred.copy()
        })

        print(f"{model_name} {params} -> {mean_rmse:.4f} ± {std_rmse:.4f}")


rf {'n_estimators': 300, 'max_features': None} -> 0.1448 ± 0.0179
rf {'n_estimators': 300, 'max_features': 'sqrt'} -> 0.1460 ± 0.0158
gbr {'n_estimators': 400, 'learning_rate': 0.05} -> 0.1336 ± 0.0211
gbr {'n_estimators': 700, 'learning_rate': 0.05} -> 0.1331 ± 0.0219
gbr {'n_estimators': 400, 'learning_rate': 0.03} -> 0.1348 ± 0.0208


In [11]:
best_row = min(cv_results, key=lambda d: d["mean_rmse"])
best_oof_pred = best_row["oof_pred"]
best_model_name = best_row["model"]        
best_params = best_row["params"] 

print("Best model:", best_model_name, best_params)

if best_model_name == "rf":    
    best_model = RandomForestRegressor(
        random_state=30,
        n_jobs=-1,
        **best_params
    )
elif best_model_name == "gbr": 
    best_model = GradientBoostingRegressor(
        random_state=30,
        **best_params
    )

Best model: gbr {'n_estimators': 700, 'learning_rate': 0.05}


In [23]:
# build mispricing table using OOF predictions (more realistic than in-sample)
actual_price = np.expm1(y.to_numpy())
pred_price   = np.expm1(best_oof_pred)

relative_error = (actual_price - pred_price) / pred_price 
abs_rel_error  = np.abs(relative_error)

mispricing = train.loc[X.index, ["Id", "Neighborhood", "OverallQual", "GrLivArea", "YearBuilt"]].copy()
mispricing["actual_price"] = actual_price  
mispricing["pred_price"]   = pred_price    
mispricing["rel_error"]    = relative_error  
mispricing["abs_rel_error"]= abs_rel_error

mispricing["valuation_flag"] = np.where(
    mispricing["rel_error"] < 0,
    "Underpriced",   # model predicts higher than market
    "Overpriced"
)

mispricing.to_csv("../analysis/mispricing_analysis.csv", index=False)

In [31]:
nb_summary = (
    mispricing
    .groupby("Neighborhood")
    .agg(
        mean_rel_error=("rel_error", "mean"),
        median_rel_error=("rel_error", "median"),
        avg_price=("actual_price", "mean"),
        count=("rel_error", "count")
    )
    .reset_index()
    .sort_values("mean_rel_error")
)

nb_summary.to_csv("../analysis/neighborhood_summary.csv", index=False)

nb_summary = nb_summary[nb_summary["count"] >= 30]

nb_summary = nb_summary.sort_values("mean_rel_error")

nb_summary.to_csv("../analysis/neighborhood_summary_filterbyCount.csv", index=False)

In [32]:
qual_summary = (
    mispricing
    .groupby("OverallQual")
    .agg(
        mean_rel_error=("rel_error", "mean"),
        median_rel_error=("rel_error", "median"),
        avg_price=("actual_price", "mean"),
        count=("rel_error", "count")
    )
    .reset_index()
    .sort_values("OverallQual")
)

qual_summary.to_csv("../analysis/quality_summary.csv", index=False)

qual_summary = qual_summary[qual_summary["count"] >= 30]

qual_summary = qual_summary.sort_values("mean_rel_error")

qual_summary.to_csv("../analysis/quality_summary_filterbyCount.csv", index=False)

In [33]:
high_qual = mispricing[mispricing["OverallQual"].isin([9, 10])]

high_qual_nb_summary = (
    high_qual
    .groupby("Neighborhood")
    .agg(
        mean_rel_error=("rel_error", "mean"),
        median_rel_error=("rel_error", "median"),
        avg_price=("actual_price", "mean"),
        count=("rel_error", "count")
    )
    .reset_index()
)

# filter sufficient sample size (e.g., count ≥ 5 for this small subset)
high_qual_nb_summary = high_qual_nb_summary[high_qual_nb_summary["count"] >= 5]

# sort by mean relative error
high_qual_nb_summary = high_qual_nb_summary.sort_values("mean_rel_error")

# save to csv
high_qual_nb_summary.to_csv("../analysis/high_quality_neighborhood_summary.csv", index=False)

In [34]:
highqual = mispricing[mispricing["OverallQual"].isin([9, 10])]
nr = highqual[highqual["Neighborhood"] == "NridgHt"].copy()  

# 1) Top 5 most "overpriced" cases in HighQual NridgHt
top5 = nr.sort_values("rel_error", ascending=False).head(5)  

cols = ["Id", "OverallQual", "Neighborhood", "actual_price", "pred_price", "rel_error", "abs_rel_error"]
print(top5[cols])

top5[cols].to_csv("../analysis/nridght_highqual_top5_overpriced.csv", index=False)


# 2) Distribution summary (helps explain mean vs median mismatch)
summary = nr["rel_error"].describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95]).to_frame().T
print(summary)
summary.to_csv("../analysis/nridght_highqual_rel_error_summary.csv", index=False)


# 3) Histogram (quick sanity check)
plt.figure()
plt.hist(nr["rel_error"], bins=20)
plt.title("NridgHt (OverallQual 9-10) rel_error distribution")
plt.xlabel("rel_error = (actual - pred) / pred")
plt.ylabel("count")
plt.tight_layout()
plt.savefig("../analysis/nridght_highqual_rel_error_hist.png", dpi=200)
plt.close()


# 4) (Optional) Save all HighQual NridgHt rows for Tableau filtering
nr[cols].to_csv("../analysis/nridght_highqual_all.csv", index=False)

        Id  OverallQual Neighborhood  actual_price     pred_price  rel_error  \
803    804            9      NridgHt      582933.0  297937.416594   0.956562   
898    899            9      NridgHt      611657.0  410016.247319   0.491787   
45      46            9      NridgHt      319900.0  262250.461583   0.219826   
1243  1244           10      NridgHt      465000.0  389836.076158   0.192809   
591    592           10      NridgHt      451950.0  394643.098482   0.145212   

      abs_rel_error  
803        0.956562  
898        0.491787  
45         0.219826  
1243       0.192809  
591        0.145212  
           count      mean       std       min        5%       25%       50%  \
rel_error   32.0  0.048761  0.206371 -0.189628 -0.118208 -0.036381 -0.008706   

                75%       95%       max  
rel_error  0.067785  0.342209  0.956562  


In [35]:
# =========================
# Underpriced asset detection (tail-based)
# =========================
# define underpricing using lower-tail threshold (e.g., bottom 5%)
q = 0.05 
thr = mispricing["rel_error"].quantile(q)   # (more negative = more underpriced)

underpriced = mispricing[mispricing["rel_error"] <= thr].copy() 
underpriced = underpriced.sort_values("rel_error")  # (most underpriced first)
# show top 10 most underpriced
cols = ["Id", "Neighborhood", "OverallQual", "GrLivArea", "YearBuilt",
        "actual_price", "pred_price", "rel_error", "abs_rel_error"]
top10_under = underpriced.head(10)[cols] 
print(f"Underpriced threshold (q={q:.2f}): rel_error <= {thr:.4f}") 
print(top10_under) 
# save full underpriced set + top10
underpriced.to_csv("../analysis/underpriced_bottom5pct.csv", index=False) 
top10_under.to_csv("../analysis/underpriced_top10.csv", index=False) 


# =========================
# Pattern check: where do underpriced cases concentrate?
# =========================
# Neighborhood concentration (filter to meaningful counts)
nb_under = (underpriced.groupby("Neighborhood")["rel_error"]
            .agg(["count", "mean", "median"])
            .sort_values("count", ascending=False)) 
print("\n[Underpriced] Neighborhood concentration:")
print(nb_under.head(15)) 
nb_under.to_csv("../analysis/underpriced_by_neighborhood.csv") 
# Quality concentration
qual_under = (underpriced.groupby("OverallQual")["rel_error"]
              .agg(["count", "mean", "median"])
              .sort_values("count", ascending=False)) 
print("\n[Underpriced] OverallQual concentration:")
print(qual_under) 
qual_under.to_csv("../analysis/underpriced_by_quality.csv") 


# =========================
# Optional: stricter rule using BOTH tail + absolute error
# =========================: keep only cases that are not just slightly negative
abs_thr = mispricing["abs_rel_error"].quantile(0.90)  # (top 10% absolute errors)
underpriced_strict = underpriced[underpriced["abs_rel_error"] >= abs_thr].copy() 
underpriced_strict = underpriced_strict.sort_values("rel_error") 
underpriced_strict.to_csv("../analysis/underpriced_strict.csv", index=False) 


Underpriced threshold (q=0.05): rel_error <= -0.1786
        Id Neighborhood  OverallQual  GrLivArea  YearBuilt  actual_price  \
523    524      Edwards           10       4676       2007      184750.0   
1298  1299      Edwards           10       5642       2008      160000.0   
30      31       IDOTRR            4       1317       1920       40000.0   
495    496       IDOTRR            4        720       1920       34900.0   
632    633       NWAmes            7       1411       1977       82500.0   
1324  1325      Somerst            8       1795       2006      147000.0   
462    463       Sawyer            5        864       1965       62383.0   
410    411      Edwards            5       1276       1958       60000.0   
968    969      OldTown            3        968       1910       37900.0   
1432  1433      OldTown            4        968       1927       64500.0   

         pred_price  rel_error  abs_rel_error  
523   733347.947758  -0.748073       0.748073  
1298  631354.5

In [21]:
thr = mispricing["rel_error"].quantile(0.90)  # top 10% underpricing
candidates = mispricing[mispricing["rel_error"] >= thr].sort_values("rel_error", ascending=False)
candidates

,Id,Neighborhood,OverallQual,GrLivArea,YearBuilt,actual_price,pred_price,rel_error,abs_rel_error
1182,1183,NoRidge,10,4476,1996,745000.0,378216.000099,0.969774,0.969774
803,804,NridgHt,9,2822,2008,582933.0,297937.416594,0.956562,0.956562
691,692,NoRidge,10,4316,1994,755000.0,407485.275595,0.852828,0.852828
688,689,StoneBr,8,1419,2007,392000.0,226956.594117,0.727203,0.727203
88,89,IDOTRR,3,1526,1915,85000.0,50078.004939,0.697352,0.697352
...,...,...,...,...,...,...,...,...,...
680,681,SawyerW,6,923,1980,143000.0,126268.649529,0.132506,0.132506
846,847,SawyerW,7,1775,1993,213000.0,188205.959535,0.131739,0.131739
149,150,BrkSide,5,1344,1936,115000.0,101627.359575,0.131585,0.131585
175,176,Edwards,6,2158,1950,243000.0,214868.838631,0.130922,0.130922
